<a href="https://colab.research.google.com/github/Raziel-JKM/CNN/blob/main/CNN_practice_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import cv2
import numpy
from tensorflow.keras import datasets, layers, models, activations, losses, optimizers, metrics

# mnist 데이터 셋을 로드
# 각각 학습셋(이미지, 라벨), 테스트 셋(이미지, 라벨)으로 구성
data_path = os.path.abspath("./mnist.npz")
(train_images, train_labels), (test_images, test_labels) = datasets.mnist.load_data(path=data_path)

# 학습과 테스트에 사용할 데이터의 수를 조정
# 참고, 원래 MNIST의 학습 이미지 셋은 60000개, 테스트 셋은 10000개 
train_cnt, test_cnt = 60000, 10000 #테스트 목적으로 적게, 원래 60000,10000
train_images, train_labels = train_images[:train_cnt], train_labels[:train_cnt]
test_images, test_labels = test_images[:test_cnt], test_labels[:test_cnt]

# MNIST의 데이터는 모두 이진 이미지이기 때문에 image의 shape이 28×28이 된다.
# 컨볼루션을 위해서는 채널수가 필요하므로 다음과 같이 이미지들을 reshaping 

train_images = train_images.reshape((train_cnt, 28, 28, 1))

# 테스트 셋 역시 사용할 갯수만큼 28x28 이진 이미지이므로 reshaping
test_images = test_images.reshape((test_cnt, 28, 28, 1))


#데이터 정규화
#서로 다른 범위의 명도값을 갖더라도 분류가 될 수 있게 하기 위해 정규화를 진행
# 픽셀 값을 0~1 사이로 정규화
train_images, test_images = train_images / 255.0, test_images / 255.0

# 모델을 구조를 선언 practice - 1 참고
model = models.Sequential() #레이어 쌓는 방식


model.add(layers.Convolution2D(32, (3, 3), #(커널의 갯수, (커널의 사이즈))
activation=activations.relu, input_shape=(28, 28, 1))) #인풋 세입은 처음에만/
model.add(layers.MaxPooling2D(pool_size=(2, 2))) # 풀사이즈(2,2 )

model.add(layers.Convolution2D(64, (3, 3), 
activation=activations.relu)) # 컨볼루션 레이어 만들기
model.add(layers.MaxPooling2D(pool_size=(2, 2))) # 풀사이즈(2,2 )
model.add(layers.Convolution2D(64, (3, 3), #인풋 세입은 처음에만
activation=activations.relu)) 



# 1차원  텐서로 변환하기
model.add(layers.Flatten()) # 1차원으로 풀어준다, 
model.add(layers.Dense(64, activation='relu'))# FC레이어 만들기 풀리 커넥티드 레이어!
model.add(layers.Dense(10, activation='softmax'))

# Model 구조를 출력합니다.
model.summary()

#모델의 컴파일
#모델에 optimizer와 손실 함수, 평가 지표를 설정
#optimizer에선 GD, SGD, Adagrad, Adam 등
#손실 함수는 MSE, categorical cross entropy 등
#마직막으로 모델의 평가지표는 accuracy, bianry accuracy 등등

# 모델을 컴파일 방법 1
adam_optimizer = optimizers.Adam()
loss_function = losses.sparse_categorical_crossentropy
metric = metrics.categorical_accuracy
model.compile(optimizer=adam_optimizer, loss=loss_function, metrics=[metric])

# 모델을 컴파일 방법 2
# model.compile(optimizer=optimizers.Adam(),
#               loss=losses.sparse_categorical_crossentropy,
#               metrics=[metrics.categorical_accuracy])


# 모델을 학습데이터로 학습

model.fit(train_images, train_labels, epochs=1)


# 모델을 평가

test_loss, test_acc = model.evaluate(test_images,  test_labels)#, verbose=2)



# 학습 결과를 출력
print("test_loss:", test_loss, "test_acc:", test_acc)

# 모델에 테스트 이미지를 넣고 예측값을 확인
test_img = cv2.imread("7.png", cv2.IMREAD_GRAYSCALE)

# 입력 이미지의 픽셀을 0~1 사이로 정규화
test_img = test_img / 255.0
row, col, channel = test_img.shape[0], test_img.shape[1], 1
confidence = model.predict(test_img.reshape((1, row, col, channel)))

for i in range(confidence.shape[1]):
    print(f"{i} 일 확률 = {confidence[0][i]}")

print(f"정답은 : {numpy.argmax(confidence, axis=1)}")